<a href="https://colab.research.google.com/github/DasBytes/three-stage-banglish-depression-classifier/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from google.colab import files

# Upload dataset
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# Dictionaries for preprocessing
slang_dict = {
    'lol': 'laughing', 'omg': 'oh my god', 'brb': 'be right back',
    'idk': 'i do not know', 'asap': 'as soon as possible',
    'r': 'are', 'u': 'you', 'thx': 'thanks', 'plz': 'please',
    'bro': 'brother', 'sis': 'sister'
}

norm_dict = {
    'valo': 'bhalo', 'kmn': 'kemon', 'kisu': 'kichu', 'kiccu': 'kichu',
    'nai': 'nei', 'n': 'na', 'pare': 'pare', 'hobe': 'hobe',
    'jabe': 'jabe', 'tumi': 'tumi', 'tmr': 'tomar'
}

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    words = text.split()
    new_words = []
    for word in words:
        word = re.sub(r'(.)\1{2,}', r'\1\1', word)
        if word in slang_dict:
            word = slang_dict[word]
        if word in norm_dict:
            word = norm_dict[word]
        new_words.append(word)

    final_words = []
    i = 0
    while i < len(new_words):
        word = new_words[i]
        if word == 'na' and i + 1 < len(new_words):
            merged = "not_" + new_words[i+1]
            final_words.append(merged)
            i += 2
        else:
            final_words.append(word)
            i += 1
    return " ".join(final_words)

df['clean_text'] = df['Sentence'].apply(clean_text)

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_text']).toarray()

le = LabelEncoder()
y = le.fit_transform(df['Category'])
y_cat = to_categorical(y)
class_names = le.classes_

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
print(f"Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Recall:    {recall_score(y_true, y_pred, average='weighted'):.4f}")
print(f"F1 Score:  {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(classification_report(y_true, y_pred, target_names=class_names))

def prediction_box(text):
    cleaned_text = clean_text(text)
    vectorized_text = tfidf.transform([cleaned_text]).toarray()
    prediction_prob = model.predict(vectorized_text, verbose=0)
    prediction_idx = np.argmax(prediction_prob)
    predicted_label = class_names[prediction_idx]
    confidence = prediction_prob[0][prediction_idx]
    return predicted_label, confidence

print("\n🔮 LIVE PREDICTION BOX")
while True:
    user_text = input("\nEnter a sentence (or type 'exit' to stop): ")
    if user_text.lower() == "exit":
        break
    label, conf = prediction_box(user_text)
    print(f"Prediction: {label}")
    print(f"Confidence: {conf:.4f}")


Saving Banglish depression dataset.csv to Banglish depression dataset (1).csv


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5332 - loss: 0.9811 - val_accuracy: 0.8435 - val_loss: 0.4849
Epoch 2/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8648 - loss: 0.3994 - val_accuracy: 0.8634 - val_loss: 0.3429
Epoch 3/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9220 - loss: 0.2189 - val_accuracy: 0.8643 - val_loss: 0.3403
Epoch 4/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9562 - loss: 0.1365 - val_accuracy: 0.8593 - val_loss: 0.3668
Epoch 5/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9783 - loss: 0.0839 - val_accuracy: 0.8543 - val_loss: 0.4137
Epoch 6/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9848 - loss: 0.0620 - val_accuracy: 0.8560 - val_loss: 0.4594
Epoch 7/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9908 - loss: 0.0373 - val_accuracy: 0.8435 - val_loss: 0.5061
Epoch 8/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9934 - loss: 0.0311 - val_accuracy: 0

KeyboardInterrupt: Interrupted by user